# 朴素贝叶斯
已知某条件概率，如何得到两个事件交换后的概率，也就是在已知P(A|B)的情况下如何求得P(B|A)。
P(A|B)示事件B已经发生的前提下，事件A发生的概率，叫做事件B发生下事件A的条件概率。其基本求
P(A|B)=P(AB)/P(B)
在生活中经常遇到这种情况：我们可以很容易直接得出P(A|B)，P(B|A)则很难直接得出，但我们更关心P(B|A)，贝叶斯定理就为我们打通从P(A|B)获得P(B|A)的道路。 
P(B|A)=P(A|B)*P(B)/P(A)

In [2]:
import numpy as np
def loadDataSet(): #导入数据
    #假设数据为最简单的6篇文章，每篇文章大概7~8个词汇左右，如下
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1] #对应上述6篇文章的分类结果，1为侮辱性，0为非侮辱性
    return postingList,classVec 

def createVocabList(dataSet):# 将所有文章中的词汇取并集汇总
    vocabSet = set([])  # 定义一个set(set存储的内容无重复)
    for document in dataSet:# 遍历导入的dataset数据，将所有词汇取并集存储至vocabSet中
        vocabSet = vocabSet | set(document) # | 符号为取并集，即获得所有文章的词汇表
    return list(vocabSet)

#该函数输入参数为词汇表及某篇文章，输出为文档向量，向量每个元素为1或0，分别表示词汇表中的单词在输入文档中是否出现；
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList) #构建一个0向量；
    for word in inputSet: # 遍历词汇表，如果文档中出现了词汇表中单词，则将输出的文档向量中对应值设为1，旨在计算各词汇出现的次数；
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1#因为上一段代码里，给的文章例子里的单词都是不重复的，如果有重复的单词的话，这段代码改写为：returnVec[vocabList.index(word)] += 1更为合适；
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec#返回向量化后的某篇文章


In [18]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix) #计算有多少篇文章
    numWords = len(trainMatrix[0]) #计算第一篇文档的词汇数
    pAbusive = sum(trainCategory)/float(numTrainDocs) #计算p(c_1)，p(c_0)=1-p(c_1)
    p0Num = np.zeros(numWords) #构建一个空矩阵，用来计算非侮辱性文章中词汇数
    p1Num = np.zeros(numWords) #构建一个空矩阵，用来计算侮辱性文章中词汇数
    p0Denom = 0.0; p1Denom = 0.0 
    for i in range(numTrainDocs): #遍历每一篇文章，来求P(w|c)
        if trainCategory[i] == 1: #判断该文章是否为侮辱性文章
            p1Num += trainMatrix[i] #累计每个词汇出现的次数
            p1Denom += sum(trainMatrix[i]) #计算所有该类别下不同词汇出现的总次数
        else:#如果该文章为非侮辱性文章
            p0Num += trainMatrix[i] 
            p0Denom += sum(trainMatrix[i])
    p1Vect = p1Num/p1Denom #计算每个词汇出现的概率P(w_i|c_1)
    p0Vect = p0Num/p0Denom #计算每个词汇出现的概率P(w_i|c_0)
    return p0Vect,p1Vect,pAbusive

def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):#vec2Classify为输入的一个向量化的新文章
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)# 由于是取对数，所以乘积变为直接求和即可，注意这里list和list是无法相乘，vec2Classify需要为array格式
    p0 = sum(vec2Classify * p0Vec) + np.log(1-pClass1)
    if(p1>p0):
        return 1
    if(p0>p1):
        return 0

In [21]:

if __name__ == '__main__':
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:     
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    # 将所有文章转化为向量，并将所有文章中的词汇append到traninMat中
    p0V,p1V,pAb = trainNB0(trainMat,listClasses)# 计算训练集的p(c_1),p(w_i|c_0),p(w_i|c_1)
    print (p0V,p1V,pAb)
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print (testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as:', classifyNB(thisDoc,p0V,p1V,pAb))



[0.04166667 0.04166667 0.04166667 0.         0.04166667 0.
 0.         0.         0.04166667 0.         0.04166667 0.
 0.04166667 0.         0.04166667 0.         0.         0.04166667
 0.04166667 0.125      0.04166667 0.04166667 0.08333333 0.04166667
 0.04166667 0.04166667 0.04166667 0.04166667 0.         0.04166667
 0.         0.04166667] [0.         0.         0.         0.05263158 0.         0.05263158
 0.05263158 0.05263158 0.         0.15789474 0.         0.10526316
 0.         0.05263158 0.10526316 0.05263158 0.05263158 0.
 0.         0.         0.         0.         0.05263158 0.
 0.         0.         0.         0.05263158 0.05263158 0.05263158
 0.05263158 0.        ] 0.5
['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as: 1


# SVM模型

支持向量机（SVM）算法被认为是文本分类中效果较为优秀的一种方法，它是一种建立在统计学习理论基础上的机器学习方法。该算法基于结构风险最小化原理，将数据集合压缩到支持向量集合，学习得到分类决策函数。这种技术解决了以往需要无穷大样本数量的问题，它只需要将一定数量的文本通过计算抽象成向量化的训练文本数据，提高了分类的精确率。

SVM 文本分类算法主要分四个步骤：文本特征提取、文本特征表示、归一化处理和文本分类。

## 文本特征提取
目前，在对文本特征进行提取时，常采用特征独立性假设来简化特征选择的过程，达到计算时间和计算质量之间的折中。一般的方法是根据文本中词汇的特征向量，通过设置特征阀值的办法选择最佳特征作为文本特征子集，建立特征模型。（特征提取前，先分词，去停用词）。

本特征提取有很多方法，其中最常用的方法是通过词频选择特征。先通过词频计算出权重，按权重从大到小排序，然后剔除无用词，这些词通常是与主题无关的，任何类的文章中都有可能大量出现的，比如“的”“是”“在”一类的词，一般在停词表中已定义好，去除这些词以后，有一个新的序列排下来，然后可以按照实际需求选取权重最高的前8个，10个或者更多词汇来代表该文本的核心内容。 
综上所述，特征项的提取步骤可以总结为：
1）对全部训练文档进行分词，由这些词作为向量的维数来表示文本；
2）统计每一类内文档所有出现的词语及其频率，然后过滤，剔除停用词和单字词；
3）统计每一类内出现词语的总词频，并取其中的若干个频率最高的词汇作为这一类别的特征词集；
4）去除每一类别中都出现的词，合并所有类别的特征词集，形成总特征词集。最后所得到的特征词集就是我们用到的特征集合，再用该集合去筛选测试集中的特征。
## 文本特征表示：
TF-IDF计算词的权值，将原来的文本信息抽象成一个向量化的样本集
## 文本分类
把样本集与训练好的模板文件进行相似度计算，若不属于该类别，则与其他类别的模板文件进行计算，直到分进相应的类别，这就是SVM 模型的文本分类方式。 



In [47]:
import numpy as np
import random
import jieba
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
    

# postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
#              ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
#              ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
#              ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
#              ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
#              ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
postingList=["my dog has flea problems help please",
             "maybe not take him to dog park stupid",
            "mr licks ate my steak how ti stop him",
             "fuck the world",
             'my dalmation is so cute I love him',
             'stop posting stupid worthless garbage']
#对应上述6篇文章的分类结果，1为侮辱性，0为非侮辱性
classVec = [0,1,0,1,0,1] 

vec = CountVectorizer()
x = vec.fit_transform(postingList)

x_train, x_test, y_train, y_test = train_test_split(x, classVec, random_state=1256)
print(x_train, x_test, y_train, y_test)

from sklearn.svm import SVC
svm = SVC(kernel='linear')
svm.fit(x_train, y_train)
y_pre= svm.predict(x_test)
print(y_pre)
print('--------------')
svm.score(x_test, y_test)

  (0, 16)	1
  (0, 3)	1
  (0, 7)	1
  (0, 4)	1
  (0, 21)	1
  (0, 8)	1
  (0, 19)	1
  (1, 25)	1
  (1, 24)	1
  (1, 20)	1
  (1, 31)	1
  (1, 6)	1
  (2, 16)	1
  (2, 9)	1
  (2, 15)	1
  (2, 12)	1
  (2, 0)	1
  (2, 23)	1
  (2, 10)	1
  (2, 28)	1
  (2, 24)	1
  (3, 3)	1
  (3, 14)	1
  (3, 17)	1
  (3, 26)	1
  (3, 9)	1
  (3, 29)	1
  (3, 18)	1
  (3, 25)	1   (0, 16)	1
  (0, 9)	1
  (0, 2)	1
  (0, 11)	1
  (0, 22)	1
  (0, 1)	1
  (0, 13)	1
  (1, 5)	1
  (1, 27)	1
  (1, 30)	1 [0, 1, 0, 1] [0, 1]
[0 1]
--------------


1.0

# LDA主题模型
在自然语言理解任务中，我们可以通过一系列的层次来提取含义——从单词、句子、段落，再到文档。在文档层面，理解文本最有效的方式之一就是分析其主题。在文档集合中学习、识别和提取这些主题的过程被称为主题建模。

所有主题模型都基于相同的基本假设：
每个文档包含多个主题；
每个主题包含多个单词。
换句话说，主题模型围绕着以下观点构建：实际上，文档的语义由一些我们所忽视的隐变量或「潜」变量管理。因此，主题建模的目标就是揭示这些潜在变量——也就是主题，正是它们塑造了我们文档和语料库的含义。

## LSA：
潜在语义分析（LSA）是主题建模的基础技术之一。其核心思想是把我们所拥有的文档-术语矩阵分解成相互独立的文档-主题矩阵和主题-术语矩阵。

LSA 模型通常用 tf-idf 得分代替文档-术语矩阵中的原始计数。直观地说，术语出现在文档中的频率越高，则其权重越大；同时，术语在语料库中出现的频率越低，其权重越大。

一旦拥有文档-术语矩阵 A，我们就可以开始思考潜在主题。问题在于：A 极有可能非常稀疏、噪声很大，并且在很多维度上非常冗余。因此，为了找出能够捕捉单词和文档关系的少数潜在主题，我们希望能降低矩阵 A 的维度。

这种降维可以使用截断 SVD 来执行。SVD，即奇异值分解，是线性代数中的一种技术。该技术将任意矩阵 M 分解为三个独立矩阵的乘积：M=U*S*V，其中 S 是矩阵 M 奇异值的对角矩阵。很大程度上，截断 SVD 的降维方式是：选择奇异值中最大的 t 个数，且只保留矩阵 U 和 V 的前 t 列。在这种情况下，t 是一个超参数，保留变换空间最重要的t维，我们可以根据想要查找的主题数量进行选择和调整。

## PLSA
pLSA，即概率潜在语义分析，采取概率方法替代 SVD 以解决问题。其核心思想是找到一个潜在主题的概率模型，该模型可以生成我们在文档-术语矩阵中观察到的数据。特别是，我们需要一个模型 P(D,W)，使得对于任何文档 d 和单词 w，P(d,w) 能对应于文档-术语矩阵中的那个条目。

# LDA

LDA 即潜在狄利克雷分布，是 pLSA 的贝叶斯版本。它使用狄利克雷先验来处理文档-主题和单词-主题分布，从而有助于更好地泛化。

考虑比较主题混合概率分布的相关例子。假设我们正在查看的语料库有着来自 3 个完全不同主题领域的文档。如果我们想对其进行建模，我们想要的分布类型将有着这样的特征：它在其中一个主题上有着极高的权重，而在其他的主题上权重不大。如果我们有 3 个主题，那么我们看到的一些具体概率分布可能会是：

混合 X：90% 主题 A，5% 主题 B，5% 主题 C
混合 Y：5% 主题 A，90% 主题 B，5% 主题 C
混合 Z：5% 主题 A，5% 主题 B，90% 主题 C
如果从这个狄利克雷分布中绘制一个随机概率分布，并对单个主题上的较大权重进行参数化，我们可能会得到一个与混合 X、Y 或 Z 非常相似的分布。我们不太可能会抽样得到这样一个分布：33％的主题 A，33％的主题 B 和 33％的主题 C。

本质上，这就是狄利克雷分布所提供的：一种特定类型的抽样概率分布法。


In [78]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from collections import Counter
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer

line = ["Youth is not a time of life; it is a state of mind. It is not a matter of rosy cheeks, red lips and supple knees. It is a matter of the will, a quality of the imagination, vigor of the emotions; it is the freshness of the deep spring of life.",
        "Youth means a temperamental predominance of courage over timidity, of the appetite for adventure over the love of ease. This often exits in a man of 60, more than a boy of 20. Nobody grows merely by the number of years; we grow old by deserting our ideas. Years may wrinkle the skin, but to give up enthusiasm wrinkles the soul. Worry, fear, self-distrust bows the heart and turns the spirit back to dust.",
        "Whether 60 or 16, there is in every human being’s heart the lure of wonders, the unfailing childlike appetite of what’s next and the joy of the game of living. In the center of your heart and my heart there is a wireless station; so long as it receives messages of beauty, hope, cheer, courage and power from men and from infinite, so long as you are young.",
        "When the aerials are down, and your spirit is covered with the snows of cynicism and the ice of pessimism, then you’ve grown old, even at 20, but as long as your aerials are up, to catch waves of optimism, there’s hope you may die young at 80."]


# line = [line]
tfidf2 = TfidfVectorizer()
re = tfidf2.fit_transform(line)
# 得到语料库all不重复的词
# print(tfidf2.get_feature_names())
# 得到每个词的对应 第几句和ID
# print(tfidf2.vocabulary_)
# 得到每个句子所对应的向量，向量里的数字的顺序是按词顺序来排列的
# 文本矩阵化
print(re.toarray())

print("文本数", len(re.toarray()))
print("单词数", len(re.toarray()[0]))
print( re)

[[0.         0.         0.         0.         0.         0.
  0.06055614 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.11604317 0.         0.         0.
  0.         0.         0.11604317 0.         0.         0.
  0.         0.         0.         0.11604317 0.         0.
  0.         0.         0.         0.         0.11604317 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.11604317 0.
  0.         0.37034427 0.36595908 0.         0.11604317 0.23208635
  0.11604317 0.         0.         0.         0.         0.
  0.23208635 0.         0.         0.         0.         0.
  0.11604317 0.         0.         0.         0.         0.23208635
  0.         0.48444913 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.11604317
  0.         0.11604317 0.11604317 0.         0.         0.
  0.         0. 

In [94]:
from gensim import corpora, models, similarities
documents= ["Youth is not a time of life it is a state of mind",
         " It is not a matter of rosy cheeks red lips and supple knees",
         " It is a matter of the will a quality of the imagination vigor of the emotions",
         "it is the freshness of the deep spring of life.",
         "Youth means a temperamental predominance of courage over timidity",
         " This often exits in a man of 60 more than a boy of 20",
         "Nobody grows merely by the number of years",
         " we grow old by deserting our ideas",
         "Years may wrinkle the skin but to give up enthusiasm wrinkles the soul",
        "there’s hope you may die young at 80."]
texts = [[word for word in document.lower().split()] for document in documents]
print(texts)
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
print(dictionary)
print(corpus)
#计算一个TF-IDF模型
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
print (tfidf.dfs)
print (tfidf.idfs)


lda = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=2)
lda.print_topics()


[['youth', 'is', 'not', 'a', 'time', 'of', 'life', 'it', 'is', 'a', 'state', 'of', 'mind'], ['it', 'is', 'not', 'a', 'matter', 'of', 'rosy', 'cheeks', 'red', 'lips', 'and', 'supple', 'knees'], ['it', 'is', 'a', 'matter', 'of', 'the', 'will', 'a', 'quality', 'of', 'the', 'imagination', 'vigor', 'of', 'the', 'emotions'], ['it', 'is', 'the', 'freshness', 'of', 'the', 'deep', 'spring', 'of', 'life.'], ['youth', 'means', 'a', 'temperamental', 'predominance', 'of', 'courage', 'over', 'timidity'], ['this', 'often', 'exits', 'in', 'a', 'man', 'of', '60', 'more', 'than', 'a', 'boy', 'of', '20'], ['nobody', 'grows', 'merely', 'by', 'the', 'number', 'of', 'years'], ['we', 'grow', 'old', 'by', 'deserting', 'our', 'ideas'], ['years', 'may', 'wrinkle', 'the', 'skin', 'but', 'to', 'give', 'up', 'enthusiasm', 'wrinkles', 'the', 'soul'], ['there’s', 'hope', 'you', 'may', 'die', 'young', 'at', '80.']]
Dictionary(73 unique tokens: ['a', 'is', 'it', 'life', 'mind']...)
[[(0, 2), (1, 2), (2, 1), (3, 1), (4

[(0,
  '0.021*"the" + 0.019*"a" + 0.017*"youth" + 0.017*"is" + 0.016*"may" + 0.016*"of" + 0.016*"temperamental" + 0.016*"means" + 0.016*"timidity" + 0.016*"predominance"'),
 (1,
  '0.022*"the" + 0.018*"by" + 0.018*"is" + 0.017*"a" + 0.017*"of" + 0.017*"matter" + 0.017*"it" + 0.016*"merely" + 0.016*"grows" + 0.016*"grow"')]